In [25]:
import hashlib
import base58

adress_from_bech32 = 'tb1qrclfyscpy5zwyuayvpvvzlzzqum72ht66490p4'
privKey_WIF_from = 'cPxyb3LtyoxLu9vAwWEKCz3gfW3BTcHEPLdZivbtsrHBvNc9ThN4'
pubKey_from = '029220b4481f0ba8d4e24e1c7ce338a360b553b07e2b4a56eb077629acdaf94090'

address_to_2 = 'tb1qwkpfmzzhezyeul4w8laaggdkecmm95rf6pcchm'


def sha256(data):
    return hashlib.sha256(data).digest()

def wif_to_hex(wif):
    # Decode WIF to bytes
    wif_bytes = base58.b58decode(wif)
    
    # Remove the version byte (0x80 for mainnet, 0xEF for testnet)
    # Remove the compression flag (0x01)
    # Remove the 4-byte checksum
    private_key_hex = wif_bytes[1:-5]
    
    return private_key_hex.hex()

In [26]:

# lets define the known variables
###  1  version
version = '02000000'

### 2 hashPrevouts
tx_hash = '55ab53696a0bb55671de6ec72508a435b256a16ed460d1e10ff51ddda56f55a1'
txid = bytes.fromhex(tx_hash)[::-1].hex()

vout = '00000000'

Prevouts = txid + vout


hashPrevouts = sha256(bytes.fromhex(sha256(bytes.fromhex(Prevouts)).hex())).hex()

print('hashPrevouts: ', hashPrevouts)

### 3 hashSequence
sequence = 'fdffffff'
hashSequence = sha256(bytes.fromhex(sha256(bytes.fromhex(sequence)).hex())).hex()

print('hashSequence: ', hashSequence)

### 4 outpoint
outpoint = txid + vout 

print('outpoint: ', outpoint)

### 5 scriptCode

# pubKey of the spender address
pubKey = bytes.fromhex(pubKey_from)
pubkey_sha256 = hashlib.sha256(pubKey).digest()
pubkey_hash = hashlib.new('ripemd160', pubkey_sha256).digest()


scriptCode = '1976a914' + pubkey_hash.hex() + '88ac'

print('scriptCode: ', scriptCode)


### 6 amount
# now lets convert 0.0047 btc that we wand to spend to satoshi
btc_value = 0.015
# Convert BTC to satoshi (1 BTC = 100,000,000 satoshis)
satoshis = int(btc_value * 100_000_000)
# Convert to 8-byte little endian format

amount = satoshis.to_bytes(8, 'little').hex().rjust(16, '0')
print(amount)

### 7 sequence
# use already defined 



### 8 hashOutputs
# scriptPubkey is OP_0 + len(hash256 of the raw_multiSig_Addr) + hash256 of the raw_multiSig_Addr

#WitnessScript_hash = sha256(bytes.fromhex(raw_multiSig_Addr)).hex()
#
#scriptPubkey = '0020' + WitnessScript_hash
#len_scriptPubkey = hex(len(scriptPubkey)//2)[2:]
scriptPubkey = '001475829d8857c8899e7eae3ffbd421b6ce37b2d069'
len_scriptPubkey = hex(len(scriptPubkey)//2)[2:]




# im not clear if i need to add len of scriptPubkey to the hashOutputs or not
Outputs = amount + len_scriptPubkey + scriptPubkey

print('outpus: ', Outputs)

hashOutputs = sha256(bytes.fromhex(sha256(bytes.fromhex(Outputs)).hex())).hex()


### 9 locktime
locktime = "00000000"

### 10  sig_hash
sig_hash = "01000000"

#  xxx
btc_in = 0.01717112

# Convert BTC to satoshi (1 BTC = 100,000,000 satoshis)
satoshis_in = int(btc_in * 100_000_000)
# Convert to 8-byte little endian format

inp_value = satoshis_in.to_bytes(8, 'little').hex().rjust(16, '0')




hashPrevouts:  6173c710d37f60d3430518f57a0cdc3a1aa14f7366a9e9fec2c84b3676912cb7
hashSequence:  caf35e5224de16efa3ccaf41070f6e7b9432b6f79551e629fca9d1c03b43bc52
outpoint:  a1556fa5dd1df50fe1d160d46ea156b235a40825c76ede7156b50b6a6953ab5500000000
scriptCode:  1976a9141e3e9243012504e273a46058c17c420737e55d7a88ac
60e3160000000000
outpus:  60e316000000000016001475829d8857c8899e7eae3ffbd421b6ce37b2d069


In [27]:
##!!!##
# for signing the TX it is important to deffirentiate between value in outputhash and value in input. 
# value in output hash is what we want to send, value in input is what we have in the input UTXO

rawTx_toSign = (
    version         #1# 4 bytes: nVersion of the transaction
    + hashPrevouts  #2# 32 bytes: SHA256 hash of all previous outputs
    + hashSequence  #3# 32 bytes: SHA256 hash of input sequence numbers
    + outpoint      #4# (32-byte hash + 4-byte little endian) 
    + scriptCode    #5# Variable length: Script of the input (often P2PKH script for P2WPKH spends)
    + inp_value        #6# 8 bytes: Value of the UTXO being spent
    + sequence      #7# 4 bytes: nSequence of the input
    + hashOutputs   #8# 32 bytes: SHA256 hash of the outputs (if SIGHASH_ALL)
    + locktime      #9# 4 bytes: nLocktime of the transaction
    + sig_hash      #10# 4 bytes: Sighash type (e.g., SIGHASH_ALL)
)

In [28]:
# now we will sign the raw transaction with our private keys

privKey = wif_to_hex(privKey_WIF_from)

print('privKey: ', privKey)


rawTx_toSign_hash = sha256(bytes.fromhex(sha256(bytes.fromhex(rawTx_toSign)).hex())).hex()

from coincurve import PrivateKey

# signing key from private key deterministically

def signTx(rawTx_toSign_hash, privKey_hex):
    bytes_to_sign = bytes.fromhex(rawTx_toSign_hash)
    privkey = PrivateKey(bytes.fromhex(privKey_hex))
    signature = privkey.sign(bytes_to_sign, hasher=None)

    # also to signale that we used SigHash all we add 01 at the end
    signature += bytes.fromhex('01')

    scriptSig = signature.hex()
    Sig_len = hex(len(signature))[2:]
    scriptSig = Sig_len + scriptSig

    return scriptSig

scriptSig1 = signTx(rawTx_toSign_hash, privKey)

print(scriptSig1)


privKey:  472240bd73e75e0f9b476ef21f0b59f34dd87149cbc80deed3d1cc6c4e0b4f96
47304402203034def7fdd51f5c1ebf81be0f5bb62c00157d56c43a50bc03a6817d763ea98e02203a6ec04d9f920060773f0f908f7777adea7bf79cc9baf04d8a7a47003fee312101


# Now we can start constructing the transaction. The structure of the transaction is very different from what has been signed (in legacy it was very similiar to what is signed, but here not)
rawTx = (
    version          #1# 4 bytes: Transaction version number
    + marker         #2# 1 byte: SegWit marker (0x00)
    + flag           #3# 1 byte: SegWit flag (0x01)
    + tx_in_count    #4# Variable length: Number of transaction inputs
    + txid           #5# 32 bytes: Transaction ID of the UTXO being spent
    + vout           #6# 4 bytes: Index of the UTXO in the transaction
    + scriptSig_empty      #7# Variable length: ScriptSig (often empty for SegWit)
    + sequence       #8# 4 bytes: Sequence number
    + tx_out_count   #9# Variable length: Number of transaction outputs
    + amount         #10# 8 bytes: Amount of satoshis to send to the recipient
    + scriptPubkey   #11# Variable length: Locking script (e.g., P2WPKH or P2WSH)
    # witness_field  Witness field, containing witness data for each input
    + witness_count
    + ScriptSig1
    + pubkey1
      
    + locktime       #13# 4 bytes: Time or block number when the transaction becomes valid
)



In [29]:
# lets define the missing variables
###  1  version (already defined)
print('version: ', version)

### 2 flag and marker (actually legacy tx expects after version, the number of inputs. 
# If it sees 00, it will know that it is SegWit with other tx structure - my understanding)


marker = '00'
flag = '01'

### 3 tx_in_count
tx_in_count = '01'

### 4 txid (already defined)
print('txid: ', txid)

### 5 vout (already defined)
print('vout: ', vout)

### 6 scriptSig (empty for segwit, will be placed in witness)
scriptSig_empty = '00'  

### 7 sequence (already defined)
print('sequence: ', sequence)

### 8 tx_out_count
tx_out_count = '01'

### 9 amount (already defined)
print('amount: ', amount)

### 10 scriptPubkey (already defined)

scriptPubkey = len_scriptPubkey + scriptPubkey
print('scriptPubkey: ', scriptPubkey)

### 11 witness count (as i understand it is the number fields in witness...will be probably clear later)
witness_count = '02'

### 12 signature (already defined)
print('scriptSig1: ', scriptSig1)

### 13 pubkey (already defined)
pubkey1 = hex(len(pubKey_from)//2)[2:] + pubKey_from

print('pubkey: ', pubkey1)

### 14 locktime (already defined)
print('locktime: ', locktime)










version:  02000000
txid:  a1556fa5dd1df50fe1d160d46ea156b235a40825c76ede7156b50b6a6953ab55
vout:  00000000
sequence:  fdffffff
amount:  60e3160000000000
scriptPubkey:  16001475829d8857c8899e7eae3ffbd421b6ce37b2d069
scriptSig1:  47304402203034def7fdd51f5c1ebf81be0f5bb62c00157d56c43a50bc03a6817d763ea98e02203a6ec04d9f920060773f0f908f7777adea7bf79cc9baf04d8a7a47003fee312101
pubkey:  21029220b4481f0ba8d4e24e1c7ce338a360b553b07e2b4a56eb077629acdaf94090
locktime:  00000000


In [30]:
rawTx = (
    version          #1# 4 bytes: Transaction version number
    + marker         #2# 1 byte: SegWit marker (0x00)
    + flag           #3# 1 byte: SegWit flag (0x01)
    + tx_in_count    #4# Variable length: Number of transaction inputs
    + txid           #5# 32 bytes: Transaction ID of the UTXO being spent
    + vout           #6# 4 bytes: Index of the UTXO in the transaction
    + scriptSig_empty      #7# Variable length: ScriptSig (often empty for SegWit)
    + sequence       #8# 4 bytes: Sequence number
    + tx_out_count   #9# Variable length: Number of transaction outputs
    + amount         #10# 8 bytes: Amount of satoshis to send to the recipient
    + scriptPubkey   #11# Variable length: Locking script (e.g., P2WPKH or P2WSH)
    # witness_field  Witness field, containing witness data for each input
    + witness_count
    + scriptSig1
    + pubkey1
      
    + locktime       #13# 4 bytes: Time or block number when the transaction becomes valid
)

print(rawTx)

02000000000101a1556fa5dd1df50fe1d160d46ea156b235a40825c76ede7156b50b6a6953ab550000000000fdffffff0160e316000000000016001475829d8857c8899e7eae3ffbd421b6ce37b2d0690247304402203034def7fdd51f5c1ebf81be0f5bb62c00157d56c43a50bc03a6817d763ea98e02203a6ec04d9f920060773f0f908f7777adea7bf79cc9baf04d8a7a47003fee31210121029220b4481f0ba8d4e24e1c7ce338a360b553b07e2b4a56eb077629acdaf9409000000000


This is the same output as produced by Bitcoin Core console:
02000000000101a1556fa5dd1df50fe1d160d46ea156b235a40825c76ede7156b50b6a6953ab550000000000fdffffff0160e316000000000016001475829d8857c8899e7eae3ffbd421b6ce37b2d0690247304402203034def7fdd51f5c1ebf81be0f5bb62c00157d56c43a50bc03a6817d763ea98e02203a6ec04d9f920060773f0f908f7777adea7bf79cc9baf04d8a7a47003fee31210121029220b4481f0ba8d4e24e1c7ce338a360b553b07e2b4a56eb077629acdaf9409000000000

-----OKEY-----